In [ ]:
from typing import Tuple, Literal

from numpy import integer, bool_, floating, ndarray

from degradation_eda.processing_stages import (
    load_data,
    mask_frames,
    estimate_poisson_uncertainty,
    correct_deadtime,
    correct_dark_current,
    correct_frame_time,
    correct_flux_and_transmission
)
from degradation_eda.utils.visualization import show_frame

In [ ]:
FILE_PATH = "../data/i22_tutorial/monoolein/i22-363079.nxs"

In [ ]:
frames: ndarray[Tuple[Literal[10], Literal[1679], Literal[1475]], integer] = load_data(FILE_PATH, "entry1/detector/data").squeeze()
show_frame(frames[0], log_z=100000)

In [ ]:
mask: ndarray[Tuple[Literal[1679], Literal[1475]], bool_] = load_data(FILE_PATH, "entry1/detector/pixel_mask")
frames = mask_frames(frames, mask)

In [ ]:
uncert_frames = estimate_poisson_uncertainty(frames)

In [ ]:
count_times: ndarray[Tuple[Literal[10]], floating] = load_data(FILE_PATH, "entry1/instrument/detector/count_time").squeeze()
dead_times: ndarray[Tuple[Literal[10]], floating] = load_data(FILE_PATH, "entry1/instrument/detector/wait_time").squeeze()

uncert_frames = correct_deadtime(uncert_frames, count_times, dead_times)

In [ ]:
uncert_frames = correct_dark_current(uncert_frames, 0.0)

In [ ]:
uncert_frames = correct_frame_time(uncert_frames, count_times, dead_times)

In [ ]:
uncert_frames = correct_flux_and_transmission(uncert_frames)

In [ ]:
show_frame(uncert_frames[0]["nominal"], log_z=100000) 